# Hybrid, Branch and Startup 



The CESM model can be initialized in one of three ways: startup, hybrid and branch. These are set with the variable **`$RUN_TYPE`** in `env_run.xml`. 

<br>

***


## 1. The three run types


#### - STARTUP

- In a startup run (the default), all model components are initialized from the basic **default initial conditions**. The coupler does not need an initial file. 



#### - BRANCH

- In a branch run, all components are initialized using **a consistent set of restart files from a previous run** (determined by the [`$RUN_REFCASE`](./run_variables.ipynb) and [`$RUN_REFDATE`](./run_variables.ipynb) variables in <font face="Courier"><strong>env_run.xml</strong></font>). 
- The case name is generally changed for a branch run, although it does not have to be. 
- In a branch run, setting [`$RUN_STARTDATE`](./run_variables.ipynb) is ignored because the model components obtain the start date from their restart datasets. Therefore, **the start date cannot be changed for a branch run**. This is the same mechanism that is used for performing a restart run (where `$CONTINUE_RUN` is set to TRUE in the <font face="Courier"><strong>env_run.xml</strong></font> file).
- Branch runs are typically used when sensitivity or parameter studies are required, or when settings for history file output streams need to be modified while still maintaining **bit-for-bit reproducibility**.




#### - HYBRID

- A hybrid run indicates that CESM is initialized more like a startup, but uses initialization datasets from a previous case. This is somewhat analogous to a branch run with relaxed restart constraints. 
- A hybrid run allows users to bring together combinations of initial/restart files from a previous case (specified by [`$RUN_REFCASE`](./run_variables.ipynb)) at a given model output date (specified by [`$RUN_REFDATE`](./run_variables.ipynb)). Unlike a branch run, the starting date of a hybrid run (specified by [`$RUN_STARTDATE`](./run_variables.ipynb)) can be modified relative to the reference case. 
- In a hybrid run, the model does not continue in a bit-for-bit fashion with respect to the reference case. The resulting climate, however, should be continuous provided that no model source code or namelists are changed in the hybrid run. 
- The atmosphere is initialized from initial condition files generated by a user-specified CESM simulation. The land, runoff, ocean and ice are initialized from restart files generated by a user-specified CESM simulation. No coupler file is needed

<br>



It is important to remember that the **`$RUN_TYPE` variable is only relevant when a run is initialized**. Let's again take a look at the [exercises](../run_length/changing_run_length.ipynb) we did in the chapter of [Changing run length](../run_length/changing_run_length.ipynb):


<br>

<div class="alert alert-info">
<strong>Evaluate your understanding</strong>

The tutorial version of the CESM model on Cheyenne simulates ~10 model years per wallclock day. The maximum wallclock request is 12 hours.
If you want to run the model for 100 years, what values should be set for `STOP_OPTION`, `STOP_N` and `RESUBMIT`? 

We worked out that for a total of 100 years, we will need 20 submissions. We can set:

```
STOP_OPTION = 'nyears', STOP_N = 5, RESUBMIT = 19
```
<br>    
    
Now, **if you want this run to be a hybrid run, what variables should you modify, and at which stage?**

</div>

<div class="alert alert-warning">
   
<details>   
   <summary>
        <img src="https://freesvg.org/img/bulb_on.png" width="20px">
       <font face="Times New Roman" color='blue' size="4">Hint!</font>
    </summary>
    
The `$RUN_TYPE` variable is only important at the initial run when `CONTINUE_RUN` is set to `FALSE`. 
    </details>
</div>

<div class="alert alert-success">
   
<details>   
   <summary>
        <img src="https://freesvg.org/img/help-browser.png" width="20px">
       <font face="Times New Roman" color='blue' size="4">Click here for the solution</font>
    </summary>

Set `RUN_TYPE='hybrid'` in <font face="Courier"><strong>env_run.xml</strong></font> before submitting the initial run.     

Bonus question: <br>
    
What happens if you set `run_type='branch'` after the 2nd resubmission when the `CONTINUE_RUN=TRUE`? <br>
    
Answer: Nothing changes, because `RUN_TYPE` is only relevant for an initial run when `CONTINUE_RUN` is set to `FALSE`! 

</details>
</div>

<br>

One more!


<div class="alert alert-info">
<strong>Evaluate your understanding</strong>


In the exercise above, if you have set `RUN_TYPE='hybrid'` and `RESUBMIT=19` before the initial run, what are the value of `CONTINUE_RUN`, `RESUBMIT`, and `RUN_TYPE` at the time of:
- the initial submission of 5 years
- the next submission of 5 years
- the 3rd run (2nd resubmission) of 5 years?
    
    
</div>

<div class="alert alert-success">
   
<details>   
   <summary>
        <img src="https://freesvg.org/img/help-browser.png" width="20px">
       <font face="Times New Roman" color='blue' size="4">Click here for the solution </font>
    </summary>
    
- **the initial submission of 5 years**: 
    
   ``` 
    CONTINUE_RUN=FALSE, RESUBMIT=19, RUN_TYPE='hybrid' 
    ```
    because when the run is first initialized, `CONTINUE_RUN=FALSE`<br>
<br>    
    
- **the next submission of 5 years**: 
    ```
    CONTINUE_RUN=TRUE, RESUBMIT=18, RUN_TYPE='hybrid'
    ```
    because `RESUBMIT>0`, `CONTINUE_RUN` will automatically switch to `TRUE` after the initial run, but `RUN_TYPE` won't change. <br>
<br> 
    
- **the 3rd run (2nd resubmission) of 5 years**: 
    ```
    CONTINUE_RUN=TRUE, RESUBMIT=17, RUN_TYPE='hybrid' 
    ```
    `CONTINUE_RUN` stays to be `TRUE`, `RESUBMIT` decreases by 1, `RUN_TYPE` won't change.
    
    
</details>
</div>

<br>

***

<br>

## 2. When to use branch vs. hybrid?

Branch and hybrid runs are useful if you have an experiment which only slightly differs from your control, but you want to make a slight modification, add history output, or start your simulation from a CESM spun-up initial state and maintaining an exact restart (which mimics what the model would do if it had kept running in the original setup).

- **Use a hybrid run**: for most applications where you do NOT need bit for bit restart. You CAN specify a new start date for your model run.

- **Use a branch run**: only for applications which require exact restart. It is typically used when sensitivity or parameter studies are required, or when settings for history file output streams need to be modified while still maintaining bit-for-bit reproducibility. You CANNOT specify a new start date for your model run. It will be assigned by the reference case ([`$RUN_REFDATE`](./run_variables.ipynb)). 

<br>

***

<br>

## 3. Restart files for branch and hybrid runs

For runs that are initialized as hybrid or branch runs, we will need restart/initial files from previous model runs (as specified by the variables, [`$RUN_REFCASE`](./run_variables.ipynb) and [`$RUN_REFDATE`](./run_variables.ipynb)). See the Chapter [Run variables related to run type](./run_variables.ipynb) for details. 


Let's take a look back at the [Chapter on restart files](../run_length/restarting.ipynb) to refresh our memory!


It is important to note that these required files must be prestaged by the user to the case run directory (`$RUNDIR`) before the model run starts. This is normally done by just copying the contents of the relevant `$RUN_REFCASE/rest/$RUN_REFDATE.00000` directory.
